In [17]:
import os
from sqlalchemy import create_engine, Column, String, Integer, Float, ForeignKey, Date, MetaData
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL
from urllib.parse import quote_plus, unquote
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Create a base class for our ORM with quoted names
Base = declarative_base(metadata=MetaData())

# Define our tables as Python classes with schema
class Författare(Base):
    __tablename__ = 'Författare'
    __table_args__ = {'schema': 'dbo'}
    ID = Column(Integer, primary_key=True)
    Förnamn = Column(String(50))
    Efternamn = Column(String(50))
    Födelsedatum = Column(Date)

class Förlag(Base):
    __tablename__ = 'Förlag'
    __table_args__ = {'schema': 'dbo'}
    ID = Column(Integer, primary_key=True)
    Namn = Column(String(100))
    Adress = Column(String(200))
    Telefon = Column(String(20))

class Böcker(Base):
    __tablename__ = 'Böcker'
    __table_args__ = {'schema': 'dbo'}
    ISBN13 = Column(String(13), primary_key=True)
    Titel = Column(String(100))
    Språk = Column(String(20))
    Pris = Column(Float)
    Utgivningsdatum = Column(Date)
    FörfattareID = Column(Integer, ForeignKey('dbo.Författare.ID'))
    FörlagID = Column(Integer, ForeignKey('dbo.Förlag.ID'))

class Butiker(Base):
    __tablename__ = 'Butiker'
    __table_args__ = {'schema': 'dbo'}
    ID = Column(Integer, primary_key=True)
    Namn = Column(String(100))
    Adress = Column(String(200))
    Stad = Column(String(100))
    Postnummer = Column(String(20))

class LagerSaldo(Base):
    __tablename__ = 'LagerSaldo'
    __table_args__ = {'schema': 'dbo'}
    ButikID = Column(Integer, ForeignKey('dbo.Butiker.ID'), primary_key=True)
    ISBN = Column(String(13), ForeignKey('dbo.Böcker.ISBN13'), primary_key=True)
    Antal = Column(Integer)

# Function to connect to the database
def connect_db():
    server_name = os.getenv('SERVER_NAME')
    database_name = os.getenv('DATABASE_NAME')
    username = os.getenv('USERNAME')
    password = os.getenv('PASSWORD')

    # Formatted connection string with SQL Server Authentication
    connection_string = (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        f"SERVER={server_name};"
        f"DATABASE={database_name};"
        f"UID={username};"
        f"PWD={password};"
    )

    # Encode the connection string
    connection_string_encoded = quote_plus(connection_string)

    # Create the URL object
    url_string = URL.create(
        "mssql+pyodbc",
        query={"odbc_connect": connection_string_encoded}
    )

    print('Connecting to database using URL string:')
    unquoted_url = unquote(str(url_string))
    print(unquoted_url, '\n')

    try:
        engine = create_engine(url_string)
        Session = sessionmaker(bind=engine)
        session = Session()
        return session
    except SQLAlchemyError as e:
        print(f"An error occurred: {e}")
        return None

# Function to search for books based on title
def search_books(session, search_term):
    try:
        results = session.query(Böcker, LagerSaldo, Butiker).\
            join(LagerSaldo, Böcker.ISBN13 == LagerSaldo.ISBN).\
            join(Butiker, LagerSaldo.ButikID == Butiker.ID).\
            filter(Böcker.Titel.like(f'%{search_term}%')).all()

        for bok, lager, butik in results:
            print(f'Titel: {bok.Titel}, Butik: {butik.Namn}, Antal: {lager.Antal}')
    except SQLAlchemyError as e:
        print(f"An error occurred: {e}")

# Main program
if __name__ == '__main__':
    session = connect_db()
    if session:
        search_term = input("Ange sökterm för boktitel: ")
        search_books(session, search_term)
        session.close()


Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3Dlocalhost%2C1433%3BDATABASE%3DBokhandel%3BUID%3Dsa%3BPWD%3D141821Tomorow%3B 

Titel: Pippi Långstrump, Butik: Bokbutiken, Antal: 10
Titel: Timming, Butik: Adlibris, Antal: 6
Titel: Igen, Butik: Bokhandel, Antal: 9
